In [1]:
#필요한 모듈을 import 합니다.
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt


#데이터 불러오기 위해 필요한 라이브러리
import os
import cv2    #open cv 파이썬 버전
from sklearn.preprocessing import LabelEncoder    #이미지의 이름을 수치화 하여 라벨링 하기 위함
from glob import glob    #디렉토리를 리스트로 불러옴


#모델에 필요한 라이브러리
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation
from tensorflow.keras.utils import to_categorical
#import h5py    #크고 복잡한 대용량 데이터를 저장하기 위한 파일 형식
                #         - 모델을 저장하기 위함


    
#L2 정규화(Ridge):모형의 복잡성은 가중치의 제곱합으로 표현
from tensorflow.keras.regularizers import l2

In [2]:
# 데이터 로드 함수
def img_load(local):
    img_paths = glob(local)
    data = []
    label = []
    
    for path in img_paths:
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (125, 125))
        
        label.append(path.split('\\')[5])        
        data.append(img)
    
    le = LabelEncoder()
    label = le.fit_transform(label)
    
    #더 높은 정확도를 위해 train 이미지를 섞음
    np.random.seed(42)
    index = np.arange(len(data))
    np.random.shuffle(index)
    
    data = np.array(data)
    data = data[index[:]]
    label = label[index[:]]
    
    return data, label

In [3]:
(x_train, t_train) = img_load('D:\\Python\\MachineLearning_FinalProject\\data\\train\\*\\*')
(x_test, t_test) = img_load('D:\\Python\\MachineLearning_FinalProject\\data\\test\\*\\*')
print('x_tarin : ', np.shape(x_train))
print('t_tarin : ', np.shape(t_train))
print('x_test : ', np.shape(x_test))
print('t_test : ', np.shape(t_test))

x_tarin :  (3372, 125, 125, 3)
t_tarin :  (3372,)
x_test :  (847, 125, 125, 3)
t_test :  (847,)


In [4]:
# 데이터 형변환
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# 데이터 스케일링
x_train_scaled = x_train/255.0
x_test_scaled = x_test/255.0
# 데이터 파티셔닝
from sklearn.model_selection import train_test_split
x_train, x_val, t_train, t_val = train_test_split(x_train_scaled, t_train, test_size=0.2, random_state=42)

In [5]:
import tensorflow as tf
(x_train, t_train) = img_load('D:\\Python\\MachineLearning_FinalProject\\data\\train\\*\\*')
(x_test, t_test) = img_load('D:\\Python\\MachineLearning_FinalProject\\data\\test\\*\\*')

In [6]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train_scaled = x_train/255
x_test_scaled = x_test/255

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_val, t_train, t_val = train_test_split(x_train_scaled, t_train, test_size=0.2, random_state=42)

In [8]:
#VGG 16은 레이어 16개로 구성
#모든 합성곱 레이어에 3X3의 작은 필터 사용
#많은 레이어 사용, 파라미터 감소
#합성곱 레이어, 풀링 레이어 번갈아가며 쌓아서 깊은 네트워크 제작
#3x3 사이즈의 작은 커널을 여러 번 반복
from tensorflow.keras.applications import vgg16 as vgg

base_model = vgg.VGG16(weights='imagenet', 
                       include_top=False, 
                       input_shape=(125, 125, 3))


58889256/58889256 [==============================] - 33s 1us/step


In [18]:
#block3_pool은 모델의 컨볼루션 레이어
#입력 이미지의 특징 추출
last = base_model.get_layer('block3_pool').output

In [19]:
from tensorflow.keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D,BatchNormalization
from tensorflow.keras import Model
# 상위 층에 분류층 추가

#GlobalAveragePooling2D은 MaxPooling2D 대신 사용
#값을 1차원으로 변환
x = GlobalAveragePooling2D()(last)
#BatchNormalization는 배치 정규화
#연산 결과가 원하는 방향의 분포대로 나오는 것이 목적
x= BatchNormalization()(x)
x = Dense(256, activation='relu')(x)
x = Dense(256, activation='relu')(x)
#Dropout은 무작위로 뉴런 제거, 과적합 방지
x = Dropout(0.6)(x)
pred = Dense(10, activation='softmax')(x)
model3 = Model(base_model.input, pred)

In [20]:
for layer in base_model.layers:
     layer.trainable = False

In [21]:
model3.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy'])

In [22]:
history = model3.fit(x=x_train, y=t_train,
                    validation_data=(x_val, t_val),
                    batch_size=32,
                    epochs=10,
                    verbose=1)
#verbose는 상세한 내용 출력 여부 조정 parameter
# 0 = 터미널에서 연산 안함
#1 = 연산 전체 보임
# 2 = loss만 보임
# 3이상 = epochs만 보임

Epoch 1/10
85/85 [==============================] - 180s 2s/step - loss: 1.7155 - accuracy: 0.4049 - val_loss: 1.5195 - val_accuracy: 0.4652
Epoch 2/10
85/85 [==============================] - 188s 2s/step - loss: 1.2485 - accuracy: 0.5640 - val_loss: 1.0838 - val_accuracy: 0.6178
Epoch 3/10
85/85 [==============================] - 193s 2s/step - loss: 1.1071 - accuracy: 0.6166 - val_loss: 0.9980 - val_accuracy: 0.6637
Epoch 4/10
85/85 [==============================] - 203s 2s/step - loss: 0.9963 - accuracy: 0.6437 - val_loss: 0.9094 - val_accuracy: 0.6667
Epoch 5/10
85/85 [==============================] - 221s 3s/step - loss: 0.8950 - accuracy: 0.6885 - val_loss: 0.8715 - val_accuracy: 0.6993
Epoch 6/10
85/85 [==============================] - 211s 2s/step - loss: 0.8427 - accuracy: 0.7052 - val_loss: 0.8582 - val_accuracy: 0.7007
Epoch 7/10
85/85 [==============================] - 191s 2s/step - loss: 0.7813 - accuracy: 0.7379 - val_loss: 0.8502 - val_accuracy: 0.6963
Epoch 8/10
85

In [23]:
model3.evaluate(x_test_scaled, t_test)

27/27 [==============================] - 48s 2s/step - loss: 0.8527 - accuracy: 0.7143


[0.8527169823646545, 0.7142857313156128]